In [1]:
import time
import pandas as pd
from datetime import date
import win32com.client as win32
import os
start_time = time.time()

## Ingresar nombre archivos

In [2]:
# Nombre archivos
Stock_Fisico = 'STOCK FISICO 15-09.xlsx'
Recurso = 'RECURSO 15-09.xlsx'
PDI = 'PDI 15-09.xlsx'
PDO = 'PDO 15-09.xlsx'
Archivo_Dia_Anterior = '2023-09-14 ON HAND COMPLETO.xlsb'

## Cargar Data

In [3]:
# Stock Fisico
df1 = pd.read_excel(Stock_Fisico ,  dtype ="string",
                    usecols = ['Tipo almacén','Ubicación','UMp sup.','UMp superior','Unidad manipulación','Documento',
                              'Número de posición','Tipo de documento','Producto','Descripción de producto','Ctd.',
                              'Propietario','Persona autorizada a disponer','Fecha EM','Hora EM','Tipo de stocks'])

# Al cargar la data, python lee las comas como puntos, por lo tanto aplicamos la funcion replace para mantenerlas como ','
df1['Ctd.'] = df1['Ctd.'].str.replace(".",",", regex = False)

In [4]:
# Recurso
df2 = pd.read_excel(Recurso, dtype = "string", 
                    usecols = ['UMp sup.','UMp superior','Unidad manipulación','Documento','Número de posición',
                              'Tipo de documento','Producto','Descripción de producto','Ctd.','Propietario',
                              'Persona autorizada a disponer','Fecha EM','Hora EM','Tipo de stocks'])

# Insertamos las columnas 'Ubicación' y 'Tipo almacén' para que nos cuadre Stock Fisico y Recurso al momento de concatenarlos
df2.insert(loc = 0, column = 'Ubicación', value = 'RECURSO')
df2.insert(loc = 0, column = 'Tipo almacén', value = 'RECURSO')

# Nos aseguramos que las dos columnas que agregamos tengan formato texto (string)
df2['Ubicación'] = df2['Ubicación'].astype('string')
df2['Tipo almacén'] = df2['Tipo almacén'].astype('string')

In [5]:
# Alinear columnas ambos archivos
columns_names = ['Tipo almacén', 'Ubicación', 'UMp sup.', 'UMp superior','Unidad manipulación', 'Documento', 
                'Número de posición','Tipo de documento', 'Producto', 'Descripción de producto', 'Ctd.','Propietario', 
                'Persona autorizada a disponer', 'Fecha EM', 'Hora EM','Tipo de stocks']

df1 = df1.reindex(columns = columns_names)
df2 = df2.reindex(columns = columns_names)

In [6]:
# Planilla de Ubicaciones-Tipo de Proceso, utilizando el On Hand completo del día anterior.
df_base2 = df3 = pd.read_excel(Archivo_Dia_Anterior , sheet_name = 'BASE 2', dtype = "string")
df3 = df_base2.loc[: , ['Ubicación','TIPO DE PROCESO']]

# Eliminamos los duplicados de la columna 'Ubicación' para no tener registros adicionales al momento de aplicar la función Join.
df3.drop_duplicates('Ubicación', inplace = True)

In [7]:
# PDI y PDO
df4 = pd.read_excel(PDI , usecols = ['Documento','Número de posición','Orden de servicio','Operación de orden de servicio'], 
                    dtype = "string")
df5 = pd.read_excel(PDO , usecols = ['Documento','Número de posición','Orden de servicio','Operación de orden de servicio'],
                    dtype = "string")

# Reordenamos Columnas
columns_names_di = ['Documento','Número de posición','Orden de servicio','Operación de orden de servicio']
df4 = df4.reindex(columns = columns_names_di)
df5 = df5.reindex(columns = columns_names_di)

In [8]:
# OnHand completo del día anterior, necesitamos las columnas CONCAT, OS y OP
df10 = pd.read_excel(Archivo_Dia_Anterior , sheet_name = 'BASE 1', engine = 'pyxlsb',
                     usecols = ['Unidad manipulación', 'Producto', 'Ctd.','OS','OP'], dtype ="string")

df10.loc[pd.isna(df10['Unidad manipulación']), 'Unidad manipulación'] = ''
df10['CONCAT'] = df10['Unidad manipulación'] + df10['Producto'] + df10['Ctd.']
df10.drop(labels = ['Unidad manipulación', 'Producto', 'Ctd.'], axis = 1, inplace = True)

## Procesamiento de datos

In [9]:
# Concatenar Stock Fisico y Recurso
df6 = pd.concat([df1,df2], axis = 0)
df6.reset_index(inplace = True)
df6.drop(labels = 'index', axis = 1, inplace = True)
df6['Identificador Unico'] = df6.index.values.tolist()

In [10]:
# Busqueda TIPO DE PROCESO según la ubicación
df3.set_index('Ubicación', inplace = True)
df6.set_index('Ubicación', inplace = True)
df6 = df6.join(df3, how = 'left')
df6.reset_index(inplace = True)

In [11]:
# Nuevas columnas CONCAT y D+I
df6.loc[pd.isna(df6['Unidad manipulación']), 'Unidad manipulación'] = ''
df6['CONCAT'] = df6['Unidad manipulación'] + df6['Producto'] + df6['Ctd.']
df6['D+I'] = df6['Documento'] + df6['Número de posición']

In [12]:
# 4. Concatenar PDI y PDO  y crear D+I para el cruce
df7 = pd.concat([df4,df5])
df7['D+I'] = df7['Documento'] + df7['Número de posición']

# Eliminamos las columnas Documento y Numero de Posición, puesto que no las necesitamos para hacer el BUSCARV
df7.drop(['Documento','Número de posición'], axis = 1, inplace = True)

In [13]:
# Busqueda Orden de Servicio y Operacion Orden de Servicio utilizando D+I
df6.set_index('D+I', inplace = True)
df7.set_index('D+I', inplace = True)
df6 = df6.join(df7, how = 'left')
df6.sort_values(by = 'Identificador Unico', inplace = True)
df6.reset_index(inplace = True)

In [14]:
# Cruce entre nuestro archivo del dia actual con el día anterior usando como valor comun los de 'CONCAT'
df10.drop_duplicates(subset = ['CONCAT'], keep = 'first', inplace = True)
df10.set_index('CONCAT', inplace = True)
df6.set_index('CONCAT', inplace = True)
df6 = df6.join(df10, how = 'left')

# Ordenamos la data según la columna 'Identificador Unico' que creamos en un principio
df6.sort_values(by = 'Identificador Unico', inplace = True)
df6.reset_index(inplace = True)

In [15]:
# Rellenar Ordenes de Servicio Vacías

# Crear Columna Auxiliares
df6['Orden de servicio_2'] = df6['Orden de servicio']

# Rellenar OS
df6.loc[pd.isna(df6['Orden de servicio']), 'Orden de servicio'] = df6.loc[pd.isna(df6['Orden de servicio']), 'OS']

# Rellenar OP
df6.loc[pd.isna(df6['Orden de servicio_2']), 'Operación de orden de servicio'] =\
df6.loc[pd.isna(df6['Orden de servicio_2']), 'OP']

# Eliminar Columna Auxiliar
df6.drop(labels = 'Orden de servicio_2' , axis = 1, inplace = True)

In [16]:
# Eliminamos las columnas que no nos sirven
df6.drop(['Identificador Unico', 'OS','OP'], axis = 1, inplace = True)

# Renombramos 'Orden de servicio' y 'Operación de orden de servicio' para que tengan el mismo nombre que el archivo original
df6.rename({'Orden de servicio': 'OS', 'Operación de orden de servicio' : 'OP'}, axis = 'columns', inplace = True)

In [17]:
# Reordenamos las columnas para que nos calze con el archivo original
column_names_oha = ['OS','OP','TIPO DE PROCESO','Tipo almacén','Ubicación','UMp sup.','UMp superior','Unidad manipulación',
                    'Documento','Número de posición','Tipo de documento','Producto','Descripción de producto','Ctd.',
                    'Propietario','Persona autorizada a disponer','Fecha EM','Hora EM','Tipo de stocks']

df6 = df6.reindex(columns = column_names_oha)

In [18]:
# Borramos los 00:00:00 de la columna FECHA EM
df6['Fecha EM'] = df6['Fecha EM'].str.replace(" 00:00:00","", regex = False)

# Colocamos el formato correcto a la columna 'FECHA EM'
df6['Fecha EM'] = pd.to_datetime(df6['Fecha EM'])
df6['Fecha EM'] = df6['Fecha EM'].dt.strftime(date_format = '%d/%m/%Y')

# Formato Correcto Columna Cantidad
df6['Ctd.'] = df6['Ctd.'].str.replace(",",".", regex = False)
df6['Ctd.'] = df6['Ctd.'].astype('float')

## Relleno de Celdas Vacías

In [19]:
# Los propietarios que no nos interesan = 0. En caso de no tener la Orden de Servicio
df6.loc[(pd.isna(df6['OS'])) & ((df6['Propietario'] != 'Y5028') & (df6['Propietario'] != 'Y5082') &\
                                 (df6['Propietario'] != 'Y5099') & (df6['Propietario'] != 'Y5059')), ['OS', 'OP']] = '0'

In [20]:
# Todo el tipo de Proceso 'BLOQUEO' = 0. Se aplica tambien para CSAR/CRE/CRC/KIT
df6.loc[(pd.isna(df6['OS'])) & (df6['TIPO DE PROCESO'] == 'BLOQUEO'), ['OS', 'OP']] = '0'

In [21]:
# Los que no tengan OS, pero cuyo producto empieze con 'SK', le colocamos OS = 0
df6.loc[(pd.isna(df6['OS'])) & (df6['Producto'].str.startswith('SK')), ['OS', 'OP']] = '0'

In [22]:
# OS = OP = 0 para la ubicación 9020-STOCKS
df6.loc[(pd.isna(df6['OS'])) & (df6['Ubicación'] == '9020-STOCK'), ['OS', 'OP']] = '0'

In [23]:
# Regularización Excedente BO = 0
df6.loc[(pd.isna(df6['OS'])) & (df6['Ubicación'] == '0290-EXCEDENTE-BO'), ['OS', 'OP']] = '0'
df6.loc[(pd.isna(df6['OS'])) & (df6['Ubicación'] == '8033-EXCEDENTE-BO'), ['OS', 'OP']] = '0'
df6.loc[(pd.isna(df6['OS'])) & (df6['Ubicación'] == '9020-EXCEDENTE-BO'), ['OS', 'OP']] = '0'

## Arreglos Finales

In [24]:
# OS y OP que quedaron vacías, las igualamos a #N/A
df6.loc[pd.isna(df6['OS']), 'OS'] = '#N/A'
df6.loc[pd.isna(df6['OP']), 'OP'] = '#N/A'

In [25]:
# Celdas en TIPO DE PROCESO, que quedaron vacías, las igualamos a #N/A
df6.loc[pd.isna(df6['TIPO DE PROCESO']), 'TIPO DE PROCESO'] = '#N/A'

In [26]:
# Lineas en TIPO DE PROCESO = ALMACEN, que no tenga documento asociado, y que tengan OS y OP distinta de cero:
# OS = OP = 0
df6.loc[(pd.isna(df6['OS']) == False) & (df6['TIPO DE PROCESO'] == 'ALMACEN') & (pd.isna(df6['Documento']) == True) &\
        (df6['OS'] != '0'), ['OS', 'OP']] = '0'

## Descarga Archivo

In [27]:
# Descargar Archivos

# 17.1 Espacio entre tablas
df_base2.rename(columns = {'Unnamed: 2':'', 'Unnamed: 6':''}, inplace = True)

# 17.2 Actualizar Nombre Archivo
today = date.today()
fecha_hoy = today.strftime("%Y-%m-%d")
Nombre_Archivo = fecha_hoy + ' ON HAND AVANCE_PG.xlsx'

# 17.3 Descargar Archivo
with pd.ExcelWriter(Nombre_Archivo) as writer:  
    df6.to_excel(writer, sheet_name='BASE 1', index = False)
    df_base2.to_excel(writer, sheet_name='BASE 2', index = False)
    
# 17.4 Abrir Instancia Excel
xlApp = win32.Dispatch('Excel.Application')
xlApp.Visible = True

# 17.5 Abrir On Hand Avance (Libro)
path = os.path.abspath(Nombre_Archivo)
wb = xlApp.Workbooks.Open(path)

# 17.6 Ajustar Columnas
wb.Worksheets(1).Activate()
wb.ActiveSheet.Columns.AutoFit()

wb.Worksheets(2).Activate()
wb.ActiveSheet.Columns.AutoFit()

True

In [28]:
end_time = time.time()
total_time = end_time - start_time

In [29]:
print(total_time / 60)

3.3701768080393473
